In [7]:
from typing_extensions import Self
import numpy as np
import os
import joblib
import torch

class QuantileModelPredictor:
    def __init__(self, model_directory):
        # Load the scaler and ensure it outputs float64
        self.scaler = joblib.load(os.path.join(model_directory, 'scaler.pkl'))

        self.weights = []
        self.biases = []

        # Load quantiles and their corresponding model weights and biases
        quantiles = np.load(os.path.join(model_directory, 'quantiles.npy'), allow_pickle=True)
        self.quantiles = quantiles

        for q in self.quantiles:
            model_path = os.path.join(model_directory, f'model_quantile_{q:.2f}.pth')
            model_info = torch.load(model_path)
            # Initialize lists to hold weights and biases for the quantile model
            model_weights = []
            model_biases = []

            # Extract weights and biases from model state dictionary and convert them to float64
            for name, param in model_info['state_dict'].items():
                if 'weight' in name:
                    model_weights.append(param.detach().numpy().astype(np.float64))
                elif 'bias' in name:
                    model_biases.append(param.detach().numpy().astype(np.float64))

            self.weights.append(model_weights)
            self.biases.append(model_biases)


    def predict_quantiles(self, inputs):
      inputs = np.array(inputs, dtype=np.float64).reshape(1, -1)
      scaled_inputs = self.scaler.transform(inputs).astype(np.float64)

      manual_outputs = []
      for i in range(len(self.quantiles)):
          x = scaled_inputs
          layers = len(self.weights[i])
          for j in range(layers):
              weights = self.weights[i][j]
              biases = self.biases[i][j]
              x = np.dot(x, weights.T) + biases
              if j < layers - 1:  # Apply ReLU to all but the last layer
                  x = np.maximum(0, x)
          manual_outputs.append(x.flatten()[0])

      return manual_outputs

# Usage example:
model_directory = '/home/yui/Downloads/read_and_play/model_output'  # Adjust the path as necessary
predictor = QuantileModelPredictor(model_directory)
inputs = [5000, 1000, 15000, 2000,5]  # Example inputs
manual_outputs = predictor.predict_quantiles(inputs)
print("qunatiles:", predictor.quantiles)
print("Manual Outputs:", manual_outputs)

qunatiles: [0.5 0.6 0.4 0.7 0.3 0.8 0.2 0.9 0.1]
Manual Outputs: [2426901354.444373, 2480358491.406657, 2307159306.8403263, 2601671413.990361, 2210020239.564946, 2872144448.735074, 2178552543.5306993, 7136673349.902651, 2097353085.123003]


In [15]:
!pip install gurobipy

import gurobipy as gp
from gurobipy import GRB
import numpy as np
import torch
import joblib
import os
class QuantileModelPredictor:
    def __init__(self, model_directory):
        self.scaler = joblib.load(os.path.join(model_directory, 'scaler.pkl'))
        quantiles = np.load(os.path.join(model_directory, 'quantiles.npy'), allow_pickle=True)
        self.models = []

        for q in sorted(quantiles):
            model_path = os.path.join(model_directory, f'model_quantile_{q:.2f}.pth')
            model_info = torch.load(model_path)
            self.models.append((model_info['weights'], model_info['state_dict']['output_layer.bias'].numpy()))

    def predict(self, inputs):
        inputs_scaled = self.scaler.transform(np.array([inputs]))
        outputs = []

        for weights, bias in self.models:
            output = inputs_scaled
            for w, b in zip(weights, bias):
                output = np.dot(output, w.T) + b
                output = np.maximum(output, 0)  # ReLU activation
            outputs.append(output.flatten()[-1])

        return outputs

def optimize(inputs, model_directory):
    model = gp.Model("Quantile_Optimization")
    x_vars = model.addVars(len(inputs), lb=0, ub=10000, name="Inputs")
    model.update()

    # Objective function placeholder
    model.setObjective(1, GRB.MAXIMIZE)

    # Constraints
    model.addConstr(gp.quicksum(x_vars[i] for i in range(len(inputs))) <= 10000, "InputSumConstraint")

    # Optimize model to find best inputs
    model.optimize()

    if model.status == GRB.OPTIMAL:
        optimized_inputs = [x_vars[i].X for i in range(len(inputs))]

        # Evaluate optimized inputs using the neural network model
        predictor = QuantileModelPredictor(model_directory)
        predictions = predictor.predict(optimized_inputs)

        # Compute weighted sum of outputs as the true objective
        weights = np.linspace(1, 2, len(predictions))  # Increasing weights as an example
        weighted_sum = sum(w * p for w, p in zip(weights, predictions))

        print("Optimized inputs:", optimized_inputs)
        print("Weighted sum of predictions:", weighted_sum)



# Example usage
inputs = [1000, 2000, 3000, 4000,5]  # initial guess
model_directory = '/home/yui/Downloads/read_and_play/model_output'  # Adjust the path as necessary
optimize(inputs, model_directory)



Defaulting to user installation because normal site-packages is not writeable
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0x0f216fb8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [1e+04, 1e+04]
Presolve removed 1 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.000000000e+00
Optimized inputs: [0.0, 0.0, 0.0, 0.0, 0.0]
Weighted sum of predictions: 21.349517563705625
